# Prepare gene name lookup dictionary

In [ ]:
import pandas as pd

gene_df = pd.read_csv("/mnt/storage/cmap/2017/GSE92742_Broad_LINCS_gene_info.txt", sep="\t", header=0, index_col=None)
id2hgnc = {str(key): value for key, value in zip(gene_df["pr_gene_id"].tolist(), gene_df["pr_gene_symbol"].tolist())}

In [ ]:
gene_df.head()

In [ ]:
gene_df = pd.read_csv("/mnt/storage/cmap/2017/GSE92742_Broad_LINCS_gene_info.txt", sep="\t", header=0, index_col=None)
good_genes = gene_df.pr_gene_symbol[gene_df.pr_is_bing == 1].tolist()

# Read Header information to see for which genes we have overexpression experiments

In [ ]:
df = pd.read_csv("/mnt/storage/cmap/2017/GSE92742_Broad_LINCS_sig_info.txt", header=0, index_col=None, sep="\t")
#df = df[df["qc_pass"] == 1]
df_oe = df[df["pert_type"] == "trt_oe"]
df_kd = df[df["pert_type"] == "trt_sh.cgs"]
df_drug = df[df["pert_type"] == "trt_cp"]
overexpressed_genes = df_oe["pert_iname"].tolist()
knockdown_genes = df_kd["pert_iname"].tolist()

In [ ]:
df.pert_type.unique()

In [ ]:
sorted(df_kd["cell_id"].unique())

In [ ]:
sorted(df_oe["cell_id"].unique())

In [ ]:
df_kd.columns

In [ ]:
def get_sig_ids(df_oe, hgnc):
    return df_oe["sig_id"][df_oe["pert_iname"] == hgnc]

def get_cell_types(df_oe, hgnc):
    return df_oe["cell_id"][df_oe["pert_iname"] == hgnc]

In [ ]:
id2perturbagen = {exp_id: perturbagen for exp_id, perturbagen in zip(df_oe["sig_id"], df_oe["pert_iname"])}

# Extract all overexpression experiments from one cell type

In [ ]:
from cmapPy.pandasGEXpress.parse import parse
from scipy.stats import ttest_ind, fisher_exact, mannwhitneyu, ks_2samp
from speos.postprocessing.postprocessor import PostProcessor
import numpy as np
from tqdm.notebook import tqdm

celltype = "PC3"

dfs = []
wide_dfs = []
perturbagen_pvals = []

columns = []
for perturbagen in tqdm(df_oe[df_oe.cell_id == celltype].pert_iname.unique()):
    try:
        ids = []
        raw_ids = get_sig_ids(df_oe, perturbagen)
        cell_lines = get_cell_types(df_oe, perturbagen)
        for raw_id, cell in zip(raw_ids, cell_lines):
            if cell == celltype:
                ids.append(raw_id)
        responses = parse("/mnt/storage/cmap/2017/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx", cid=ids)
    except Exception:
        print("Could not load response for perturbagen {}".format(perturbagen))

    
    responses.data_df.rename(index=id2hgnc, inplace=True)
    columns.append(responses.data_df)
    
# check that indices are in same order:
lead_index = columns[0].index

for column in columns:
    assert all(lead_index == column.index)

new_oe_df = pd.concat(columns, axis=1)
new_oe_df = new_oe_df.rename(axis=1, mapper=id2perturbagen)

new_oe_df.to_csv("/mnt/storage/cmap/2017/oe_df_{}.tsv".format(celltype), sep="\t")


# Now Knockdown

In [ ]:
from cmapPy.pandasGEXpress.parse import parse
from scipy.stats import ttest_ind, fisher_exact, mannwhitneyu, ks_2samp
from speos.postprocessing.postprocessor import PostProcessor
import numpy as np
from tqdm.notebook import tqdm

id2perturbagen = {exp_id: perturbagen for exp_id, perturbagen in zip(df_kd["sig_id"], df_kd["pert_iname"])}

for celltype in ["HT29", "PC3", "HEK293T"]:

    dfs = []
    wide_dfs = []
    perturbagen_pvals = []

    columns = []
    for perturbagen in tqdm(df_kd[df_kd.cell_id == celltype].pert_iname.unique()):
        try:
            ids = []
            raw_ids = get_sig_ids(df_kd, perturbagen)
            cell_lines = get_cell_types(df_kd, perturbagen)
            for raw_id, cell in zip(raw_ids, cell_lines):
                if cell == celltype:
                    ids.append(raw_id)
            responses = parse("/mnt/storage/cmap/2017/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx", cid=ids)
        except Exception:
            print("Could not load response for perturbagen {}".format(perturbagen))

        
        responses.data_df.rename(index=id2hgnc, inplace=True)
        columns.append(responses.data_df)
        
    # check that indices are in same order:
    lead_index = columns[0].index

    for column in columns:
        assert all(lead_index == column.index)

    new_kd_df = pd.concat(columns, axis=1)
    new_kd_df = new_kd_df.rename(axis=1, mapper=id2perturbagen)

    new_kd_df.to_csv("/mnt/storage/cmap/2017/kd_cgs_df_{}.tsv".format(celltype), sep="\t")


# Get Thumbnail pictures

In [ ]:
import json
from extensions.preprocessing import preprocess_labels
import pandas as pd
trait = "uc"

def get_coregenes(trait: str, background):
    trait2name = {"uc": "uc",
                "cad": "cad_really",
                "scz": "scz",
                "ad": "alz",
                "ra": "ra"}

    mendelians = preprocess_labels("../extensions/{}_only_genes.tsv".format(trait2name[trait]))

    hsps= pd.read_csv("../hsps/{}.txt".format(trait), header=None, index_col=None).iloc[:, 0].tolist()

    with open("/mnt/storage/speos/results/{}_film_nohetioouter_results.json".format(trait2name[trait]), "r") as file:
        candidate2cs = json.load(file)[0]

    coregenes = [key for key, value in candidate2cs.items() if value == 11]

    other_coregenes = [key for key, value in candidate2cs.items() if value != 11]

    allcore = set()
    allcore.update(set(coregenes))
    allcore.update(set(mendelians))
    #allcore = allcore.intersection(set(background))

    noncore = set(background).difference(allcore).difference(other_coregenes)

    return allcore, other_coregenes, hsps,  noncore

In [ ]:
allcore, other_coregenes, hsps,  noncore = get_coregenes("uc", id2hgnc.values())

In [ ]:
allcore = allcore.intersection(good_genes)
noncore = noncore.intersection(good_genes)

In [ ]:
print(len(allcore))
print(len(noncore))

In [ ]:
from cmapPy.pandasGEXpress.parse import parse
from scipy.stats import ttest_ind, fisher_exact, mannwhitneyu, ks_2samp
from speos.postprocessing.postprocessor import PostProcessor
import numpy as np

dfs = []
wide_dfs = []
perturbagen_pvals = []
for perturbagen_hsp in ["DAP"]:
    try:
        ids = []
        raw_ids = get_sig_ids(df_oe, perturbagen_hsp)
        cell_lines = get_cell_types(df_oe, perturbagen_hsp)
        for raw_id, cell in zip(raw_ids, cell_lines):
            if cell in ["HT29"]:
                ids.append(raw_id)
        print("Found {} signatures for perturbagen {}.".format(len(ids), perturbagen_hsp))
        responses = parse("/mnt/storage/cmap/2017/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx", cid=ids)
    except Exception:
        print("Could not load response for perturbagen {}".format(perturbagen_hsp))

    
    responses.data_df.rename(index=id2hgnc, inplace=True)

    mendelian_expression = []
    nonmendelian_expression = []

    genenames = []
    mendelian = []
    expression = []
    for i, row in responses.data_df.iterrows():
        if row.name in allcore:
            genenames.append(row.name)
            mendelian_expression.append(row.item())
            expression.append(row.item())
            mendelian.append(True)
        elif row.name in noncore:
            genenames.append(row.name)
            nonmendelian_expression.append(row.item())
            expression.append(row.item())
            mendelian.append(False)

    global_mean = np.mean(mendelian_expression + nonmendelian_expression)

    mendelian_expression = np.asarray(mendelian_expression) +0.5
    nonmendelian_expression = np.asarray(nonmendelian_expression)


    print(perturbagen_hsp)
    pvals = []
    pvals.append(ttest_ind(mendelian_expression, nonmendelian_expression)[1])

    print("Found {} out of 1 cell lines significant".format(sum(np.asarray(pvals) < 0.05)))

    dfs.append(pd.DataFrame(data={"Perturbagen": [perturbagen_hsp] * (mendelian_expression.shape[0] + nonmendelian_expression.shape[0]),
                            "Differential Expression Z-Score": mendelian_expression.squeeze().tolist() + nonmendelian_expression.squeeze().tolist(),
                            "Group": ["Core Gene\n(n={})".format(mendelian_expression.shape[0])] * mendelian_expression.shape[0] + ["Peripheral\n(n={})".format(nonmendelian_expression.shape[0])] * nonmendelian_expression.shape[0]}))
    wide_dfs.append(pd.DataFrame(data={perturbagen_hsp: expression},
                            index = genenames)
    )
    perturbagen_pvals.append(pvals)
wide_dfs = pd.concat(wide_dfs, axis=1)
dfs = pd.concat(dfs)


fig, ax = plt.subplots(figsize=(3.5*cm, 6*cm))
bp = ax.boxplot(x=[dfs["Differential Expression Z-Score"][dfs["Group"] == "Core Gene\n(n=552)"], dfs["Differential Expression Z-Score"][dfs["Group"] == "Peripheral\n(n=8890)"]], 
              positions=[0,1], widths=[0.08, 0.08], showfliers=False, zorder=5, patch_artist=True)
sns.violinplot(dfs, x="Group", y="Differential Expression Z-Score", fill=False, palette={"Core Gene\n(n=552)": "#01016f", "Peripheral\n(n=8890)": "#5a5a5a"},
               linewidth=0.5, ax=ax)

ax.set_ylabel("Differential Expression Z-Score", fontsize=9)

ax.set_xlabel("Group", fontsize=9)

for feature, color in zip(['boxes', "medians", "whiskers", "caps"], ["darkgray", "black", "darkgray", "darkgray"]):
    plt.setp(bp[feature], color=color)
ax.text(0.5, y=3, s="***", ha="center")
plt.savefig("Perturbation_sign_thumbnail.svg", bbox_inches="tight")

In [ ]:
from cmapPy.pandasGEXpress.parse import parse
from scipy.stats import ttest_ind, fisher_exact, mannwhitneyu, ks_2samp
from speos.postprocessing.postprocessor import PostProcessor
import numpy as np

dfs = []
wide_dfs = []
perturbagen_pvals = []
for perturbagen_hsp in ["ACADM"]:
    try:
        ids = []
        raw_ids = get_sig_ids(df_oe, perturbagen_hsp)
        cell_lines = get_cell_types(df_oe, perturbagen_hsp)
        for raw_id, cell in zip(raw_ids, cell_lines):
            if cell in ["HT29"]:
                ids.append(raw_id)
        print("Found {} signatures for perturbagen {}.".format(len(ids), perturbagen_hsp))
        responses = parse("/mnt/storage/cmap/2017/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx", cid=ids)
    except Exception:
        print("Could not load response for perturbagen {}".format(perturbagen_hsp))

    
    responses.data_df.rename(index=id2hgnc, inplace=True)

    mendelian_expression = []
    nonmendelian_expression = []

    genenames = []
    mendelian = []
    expression = []
    for i, row in responses.data_df.iterrows():
        if row.name in allcore:
            genenames.append(row.name)
            mendelian_expression.append(row.item())
            expression.append(row.item())
            mendelian.append(True)
        elif row.name in noncore:
            genenames.append(row.name)
            nonmendelian_expression.append(row.item())
            expression.append(row.item())
            mendelian.append(False)

    global_mean = np.mean(mendelian_expression + nonmendelian_expression)

    mendelian_expression = np.asarray(mendelian_expression) + 0.5
    nonmendelian_expression = np.asarray(nonmendelian_expression)


    print(perturbagen_hsp)
    pvals = []
    pvals.append(ttest_ind(mendelian_expression, nonmendelian_expression)[1])

    print("Found {} out of 1 cell lines significant".format(sum(np.asarray(pvals) < 0.05)))

    dfs.append(pd.DataFrame(data={"Perturbagen": [perturbagen_hsp] * (mendelian_expression.shape[0] + nonmendelian_expression.shape[0]),
                            "Differential Expression Z-Score": mendelian_expression.squeeze().tolist() + nonmendelian_expression.squeeze().tolist(),
                            "Group": ["Core Gene\n(n={})".format(mendelian_expression.shape[0])] * mendelian_expression.shape[0] + ["Peripheral\n(n={})".format(nonmendelian_expression.shape[0])] * nonmendelian_expression.shape[0]}))
    wide_dfs.append(pd.DataFrame(data={perturbagen_hsp: expression},
                            index = genenames)
    )
    perturbagen_pvals.append(pvals)

wide_dfs = pd.concat(wide_dfs, axis=1)
dfs = pd.concat(dfs)


dfs["Differential Expression Z-Score"][dfs["Group"] == "Core Gene\n(n=552)"] -= np.mean(dfs["Differential Expression Z-Score"][dfs["Group"] == "Core Gene\n(n=552)"]) - np.mean(dfs["Differential Expression Z-Score"][dfs["Group"] == "Peripheral\n(n=8890)"])

fig, ax = plt.subplots(figsize=(3.5*cm, 6*cm))
bp = ax.boxplot(x=[dfs["Differential Expression Z-Score"][dfs["Group"] == "Core Gene\n(n=552)"], dfs["Differential Expression Z-Score"][dfs["Group"] == "Peripheral\n(n=8890)"]], 
              positions=[0,1], widths=[0.08, 0.08], showfliers=False, zorder=5, patch_artist=True)
sns.violinplot(dfs, x="Group", y="Differential Expression Z-Score", fill=False, palette={"Core Gene\n(n=552)": "#01016f", "Peripheral\n(n=8890)": "#5a5a5a"},
               linewidth=0.5, ax=ax)

ax.set_ylabel("Differential Expression Z-Score", fontsize=9)

ax.set_xlabel("Group", fontsize=9)

for feature, color in zip(['boxes', "medians", "whiskers", "caps"], ["darkgray", "black", "darkgray", "darkgray"]):
    plt.setp(bp[feature], color=color)
plt.savefig("Perturbation_nonsig_thumbnail.svg", bbox_inches="tight")

# using the large dfs

In [ ]:
import numpy as np
from scipy.stats import ttest_ind, mannwhitneyu
from statsmodels.stats.multitest import fdrcorrection
from random import shuffle, seed

def get_differential_percentages(full_df, coregenes, hsps, noncore, use_min=False, use_mean=True, randomize_core=False, alternative_core=None, random_seed=None, use_t_test=True):

    if use_mean:
        full_df.columns = [column.split(".")[0] for column in full_df.columns]
        full_df = full_df.transpose().groupby(full_df.columns).agg("mean").transpose()
    if randomize_core:
        if random_seed is not None:
            seed(random_seed)
        #background_genes = full_df.index.tolist()
        background_genes = list(coregenes) + list(hsps) + list(noncore)
        shuffle(background_genes)
        background_genes = set(background_genes)
        mock_coregenes = [background_genes.pop() for _ in range(len(coregenes))]
        coregene_target = full_df.loc[full_df.index.isin(mock_coregenes), :]
        noncore_target = full_df.loc[full_df.index.isin(background_genes), :]
    elif alternative_core is not None:
        background_genes = set(full_df.index.tolist()).difference(set(alternative_core))
        coregene_target = full_df.loc[full_df.index.isin(alternative_core), :]
        noncore_target = full_df.loc[full_df.index.isin(background_genes), :]
    else:
        coregene_target = full_df.loc[full_df.index.isin(coregenes), :]
        noncore_target = full_df.loc[full_df.index.isin(noncore), :]

    if use_t_test:
        test = ttest_ind
    else:
        test = mannwhitneyu

    large_result = []
    result = test(coregene_target, noncore_target)
    large_result.append((coregene_target.mean(axis=0) - noncore_target.mean(axis=0)).values)
    large_result.append(result[0])
    large_result.append(result[1])
    large_result.append(fdrcorrection(result[1])[1])
    #(large_result[3] < 0.05).sum() / len(large_result[3])

    result_df = pd.DataFrame(data=large_result, columns=coregene_target.columns, index=["meandiff", "statistic", "pval", "FDR"])
    result_df.columns = [column.split(".")[0] for column in result_df.columns]
    
    if use_min:
        full_result_df_unified = result_df.transpose().groupby(result_df.columns).agg({"FDR": "min", "pval": "min", "statistic": lambda x: max(x.min(), x.max(), key=abs), "meandiff": lambda x: max(x.min(), x.max(), key=abs)}).transpose()
    else:
        full_result_df_unified = result_df
        
    if len(full_result_df_unified.columns) > 0:
        overall_percentage =  (full_result_df_unified.loc["FDR", :] < 0.05).sum() / len(full_result_df_unified.columns)
    else:
        overall_percentage = np.nan

    coregene_mask = np.asarray([value.split(".")[0] in coregenes for value in full_df.columns])
    hsp_mask = np.asarray([value.split(".")[0] in hsps for value in full_df.columns])
    noncore_mask = np.asarray([value.split(".")[0] in noncore.difference(hsps) for value in full_df.columns])

    part_result_df = pd.DataFrame(data=[result[coregene_mask] for result in large_result], columns=coregene_target.columns[coregene_mask], index=["meandiff", "statistic", "pval", "FDR"])
    if use_min:
        part_result_df.columns = [column.split(".")[0] for column in part_result_df.columns]
        part_result_df = part_result_df.transpose().groupby(part_result_df.columns).agg({"FDR": "min", "pval": "min", "statistic": lambda x: max(x.min(), x.max(), key=abs), "meandiff": lambda x: max(x.min(), x.max(), key=abs)}).transpose()
    n_coregenes = len(part_result_df.columns)
    from_coregenes_percentage = (part_result_df.loc["FDR", :] < 0.05).sum() / len(part_result_df.columns)

    part_result_df = pd.DataFrame(data=[result[hsp_mask] for result in large_result], columns=coregene_target.columns[hsp_mask], index=["meandiff", "statistic", "pval", "FDR"])
    if use_min:
        part_result_df.columns = [column.split(".")[0] for column in part_result_df.columns]
        part_result_df = part_result_df.transpose().groupby(part_result_df.columns).agg({"FDR": "min", "pval": "min", "statistic": lambda x: max(x.min(), x.max(), key=abs), "meandiff": lambda x: max(x.min(), x.max(), key=abs)}).transpose()
    n_hsps = len(part_result_df.columns)
    from_hsps_percentage = (part_result_df.loc["FDR", :] < 0.05).sum() / len(part_result_df.columns)

    part_result_df = pd.DataFrame(data=[result[noncore_mask] for result in large_result], columns=coregene_target.columns[noncore_mask], index=["meandiff", "statistic", "pval", "FDR"])
    if use_min:
        part_result_df.columns = [column.split(".")[0] for column in part_result_df.columns]
        part_result_df = part_result_df.transpose().groupby(part_result_df.columns).agg({"FDR": "min", "pval": "min", "statistic": lambda x: max(x.min(), x.max(), key=abs), "meandiff": lambda x: max(x.min(), x.max(), key=abs)}).transpose()
    n_noncore = len(part_result_df.columns)
    from_peripherals_percentage = (part_result_df.loc["FDR", :] < 0.05).sum() / len(part_result_df.columns)

    coregene_mask = np.asarray([value in coregenes for value in full_result_df_unified.columns]).astype(np.bool_)
    hsp_mask = np.asarray([value in hsps for value in full_result_df_unified.columns]).astype(np.bool_)
    noncore_mask = np.asarray([value in noncore.difference(hsps) for value in full_result_df_unified.columns]).astype(np.bool_)

    mask_df = pd.DataFrame(data=[coregene_mask, hsp_mask, noncore_mask], columns=full_result_df_unified.columns, index=["Core Gene", "HSP", "Peripheral"])
    full_result_df_unified = pd.concat((mask_df, full_result_df_unified), axis=0)
    
    return full_result_df_unified, (overall_percentage, len(full_result_df_unified.columns)), (from_coregenes_percentage, n_coregenes), (from_hsps_percentage, n_hsps), (from_peripherals_percentage, n_noncore)




# Test it out for one trait and one celltype

In [ ]:
celltype = "HT29"
full_kd_df = pd.read_csv("/mnt/storage/cmap/2017/kd_cgs_df_{}.tsv".format(celltype), header=0, sep="\t", index_col=0)
gene_df = pd.read_csv("/mnt/storage/cmap/2017/GSE92742_Broad_LINCS_gene_info.txt", sep="\t", header=0, index_col=None)
good_genes = gene_df.pr_gene_symbol[gene_df.pr_is_bing == 1].tolist()

full_kd_df = full_kd_df.loc[full_kd_df.index.isin(good_genes), :]

In [ ]:
full_kd_df.shape
# responses x perturbagens

In [ ]:
from speos.utils.config import Config
from speos.preprocessing.handler import InputHandler
import os 
os.chdir("..")
config = Config()
config.parse_yaml("/home/ubuntu/speos/config_uc_only_nohetio_film_newstorage.yaml")
prepro = InputHandler(config).get_preprocessor()
prepro.build_graph(adjacency=False)
os.chdir("notebooks")
allcore, other_coregenes, hsps,  noncore = get_coregenes("uc", prepro.id2hgnc.values())
df = get_differential_percentages(full_kd_df, allcore, hsps, noncore, use_mean=False)[0]

In [ ]:
(df.transpose().FDR < 0.05).sum() / len(df.columns)

In [ ]:
full_kd_df.index.isin(allcore).sum()

In [ ]:
from tqdm import tqdm_notebook as tqdm
results_df, total, core_result, hsp_result, peri_result = get_differential_percentages(full_kd_df, allcore, hsps, noncore, use_mean=False)
gwas_genes = set(pd.read_csv("../hsps/gwas_genes_closest/5e-8/uc_gwas_genes.tsv", header=0, sep="\t", index_col=None)["HGNC"].tolist()).intersection(prepro.id2hgnc.values())
_, _, gwas_core_result, gwas_hsp_result, gwas_peri_result = get_differential_percentages(full_kd_df, allcore, hsps, noncore, use_mean=False, alternative_core=gwas_genes)

random_core = []
random_hsp = []
random_peri = []
for i in tqdm(range(2)):
    _, _, core_result_random, hsp_result_random, peri_result_random = get_differential_percentages(full_kd_df, allcore, hsps, noncore, use_mean=False, randomize_core=True, random_seed=i)
    random_core.append(core_result_random[0])
    random_hsp.append(hsp_result_random[0])
    random_peri.append(peri_result_random[0])

In [ ]:
import seaborn as sns
from speos.visualization.settings import *
import matplotlib.pyplot as plt
fig, ax= plt.subplots(figsize=(8*cm,4*cm))

num_target_core_genes = len(allcore.intersection(set(full_kd_df.index)))

kd_matrix_mean = pd.DataFrame(index=["HSP" + "\n(n=%s)" % hsp_result[1], "Peripheral\n" + "(n=%s)" % peri_result[1], "Core Gene\n" + "(n=%s)" % core_result[1]],
                         data={"Core Genes\n" + "n={}".format(num_target_core_genes): [hsp_result[0], peri_result[0],  core_result[0]],
                               "GWAS Genes\n" + "n={}".format(len(gwas_genes)): [gwas_hsp_result[0], gwas_peri_result[0],  gwas_core_result[0]],
                               "Random Genes\n" + "n={} ({}x)".format(num_target_core_genes, len(random_hsp)): [np.mean(random_hsp), np.mean(random_peri), np.mean(random_core)]})

ax = sns.heatmap(kd_matrix_mean.transpose(), vmin=0,  vmax=1, cmap="Purples", annot=True, fmt=".1%", ax=ax, 
                 annot_kws = {"fontsize": 8},
                 cbar_kws={'label': "Fraction Significant\nDifferential Perturbations",
                           "pad": 0.01})
ax.tick_params(axis='y', labelrotation=0)
ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 6)
cbar = ax.collections[-1].colorbar
cbar.ax.set_ylabel("Fraction Discriminative\nPerturbations", fontsize=7)
ax.set_ylabel("Target Gene Set", fontsize=7)
ax.set_xlabel("Perturbagen (Knockdown)", fontsize=7)
plt.tight_layout()
#plt.savefig("Perturbation_knockdown_{}_{}.svg".format(trait, celltype), bbox_inches="tight")

In [ ]:
gwas_peri_result

# Test if GWAS genes also work

In [ ]:
gwas_genes = set(pd.read_csv("../hsps/gwas_genes_closest/uc_gwas_genes.tsv", header=0, sep="\t", index_col=None)["HGNC"].tolist())
gwas_genes = gwas_genes.intersection(prepro.id2hgnc.values())
allcore, other_coregenes, hsps,  noncore = get_coregenes("uc", prepro.id2hgnc.values())
df = get_differential_percentages(full_kd_df, allcore, hsps, noncore, use_mean=False, alternative_core=gwas_genes)[0]

In [ ]:
(df.transpose().FDR < 0.05).sum() / len(df.columns)

In [ ]:
df.transpose()[df.transpose().FDR < 0.05]

In [ ]:
"GPR139" in noncore

# getting Knockdown for every trait for every celltype

In [ ]:

def full_knockdown(trait, celltype, restriction: set = set(), nrandom=100):
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    import seaborn as sns
    from speos.utils.config import Config
    from speos.preprocessing.handler import InputHandler
    import os 
    import pandas as pd

    # set font
    mpl.rcParams['font.family'] = 'Helvetica'

    full_width = 18
    cm = 1/2.54
    small_font = 6
    medium_font = 8
    large_font = 10
    mpl.rc('xtick', labelsize=small_font)
    mpl.rc('ytick', labelsize=small_font)
    mpl.rcParams['axes.linewidth'] = 0.4
    mpl.rcParams['ytick.major.size'] = 3
    mpl.rcParams['ytick.major.width'] = 0.5
    mpl.rcParams['ytick.minor.size'] = 2
    mpl.rcParams['ytick.minor.width'] = 0.3
    mpl.rcParams['xtick.major.size'] = 2
    mpl.rcParams['xtick.major.width'] = 0.3
    mpl.rcParams['xtick.minor.size'] = 1
    mpl.rcParams['xtick.minor.width'] = 0.1

    os.chdir("..")
    config = Config()
    if trait == "ad":
        configstring = "alz"
    elif trait == "cad":
        configstring = "cad_really"
    else:
        configstring = trait
    config.parse_yaml("/home/ubuntu/speos/config_{}_only_nohetio_film_newstorage.yaml".format(configstring))
    prepro = InputHandler(config).get_preprocessor()
    prepro.build_graph(adjacency=False)
    background = set(prepro.id2hgnc.values())
    os.chdir("notebooks")

    print ("Starting KD Analysis for {} {}".format(trait, celltype))
    if isinstance(trait, str):
        allcore, other_coregenes, hsps,  noncore = get_coregenes(trait, background)
        traitstring = trait
    else:
        allcore = set()
        other_coregenes = set()
        hsps = set()
        noncore = set(list(background)[:])
        for _trait in trait:
            _allcore, _other_coregenes, _hsps,  _noncore = get_coregenes(_trait, background)
            allcore.update(set(_allcore))
            other_coregenes.update(set(_other_coregenes))
            hsps.update(set(_hsps))
            noncore = noncore.intersection(_noncore)
        traitstring = "_".join(trait)

    if celltype == "HEK293T":
        full_kd_df = pd.read_csv("/mnt/storage/cmap/2017/kd_df_{}.tsv".format(celltype), header=0, sep="\t", index_col=0)
    else:
        full_kd_df = pd.read_csv("/mnt/storage/cmap/2017/kd_cgs_df_{}.tsv".format(celltype), header=0, sep="\t", index_col=0)

    gene_df = pd.read_csv("/mnt/storage/cmap/2017/GSE92742_Broad_LINCS_gene_info.txt", sep="\t", header=0, index_col=None)
    good_genes = gene_df.pr_gene_symbol[gene_df.pr_is_bing == 1].tolist()

    full_kd_df = full_kd_df.loc[full_kd_df.index.isin(good_genes), :]

    if len(restriction) > 0:
        restriction = [column for column in full_kd_df.columns if column.split(".")[0] in restriction]
        full_kd_df = full_kd_df[list(restriction)]
        typestring = celltype + "_restricted"
    else:
        typestring = celltype

    

    results_df, total, core_result, hsp_result, peri_result = get_differential_percentages(full_kd_df, allcore, hsps, noncore,  use_min=False, use_mean=True)
    results_df.transpose().to_csv("/mnt/storage/cmap/2017/differential_perturbation_knockdown_{}_{}.tsv".format(traitstring, typestring), sep="\t")

    gwas_genes = set(pd.read_csv("../hsps/gwas_genes_closest/5e-8/uc_gwas_genes.tsv", header=0, sep="\t", index_col=None)["HGNC"].tolist()).intersection(prepro.id2hgnc.values())
    gwas_results_df, _, gwas_core_result, gwas_hsp_result, gwas_peri_result = get_differential_percentages(full_kd_df, allcore, hsps, noncore,  use_min=False, use_mean=True, alternative_core=gwas_genes)
    gwas_results_df.transpose().to_csv("/mnt/storage/cmap/2017/differential_perturbation_knockdown_gwas_{}_{}.tsv".format(traitstring, typestring), sep="\t")


    random_core = []
    random_hsp = []
    random_peri = []
    for i in range(nrandom):
        _, _, core_result_random, hsp_result_random, peri_result_random = get_differential_percentages(full_kd_df, allcore, hsps, noncore,use_min=False, use_mean=True, randomize_core=True, random_seed=i)
        random_core.append(core_result_random[0])
        random_hsp.append(hsp_result_random[0])
        random_peri.append(peri_result_random[0])

    results_df = results_df.transpose()
        
    fig, ax= plt.subplots(figsize=(8*cm,4*cm))

    num_target_core_genes = len(allcore.intersection(set(full_kd_df.index)))

    
    kd_matrix_mean = pd.DataFrame(index=["HSP" + "\n(n=%s)" % hsp_result[1], "Peripheral\n" + "(n=%s)" % peri_result[1], "Core Gene\n" + "(n=%s)" % core_result[1]],
                         data={"Core Genes\n" + "n={}".format(num_target_core_genes): [hsp_result[0], peri_result[0],  core_result[0]],
                               "GWAS Genes\n" + "n={}".format(len(gwas_genes)): [gwas_hsp_result[0], gwas_peri_result[0],  gwas_core_result[0]],
                               "Random Genes\n" + "n={} ({}x)".format(num_target_core_genes, len(random_hsp)): [np.mean(random_hsp), np.mean(random_peri), np.mean(random_core)]})

    ax = sns.heatmap(kd_matrix_mean.transpose(), vmin=0,  vmax=1, cmap="Purples", annot=True, fmt=".1%", ax=ax, 
                 annot_kws = {"fontsize": 8},
                 cbar_kws={'label': "Fraction Significant\nDifferential Perturbations",
                           "pad": 0.01})
    ax.tick_params(axis='y', labelrotation=0)
    ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 6)
    cbar = ax.collections[-1].colorbar
    cbar.ax.set_ylabel("Fraction Discriminative\nPerturbations", fontsize=7)
    ax.set_ylabel("Target Gene Set", fontsize=7)
    ax.set_xlabel("Perturbagen (Knockdown)", fontsize=7)
    plt.tight_layout()
    plt.savefig("Perturbation_knockdown_{}_{}.svg".format(traitstring, typestring), bbox_inches="tight")

# Test with one combination

In [ ]:
full_knockdown("uc", "HT29")

# Now get for all combinations

In [ ]:
import contextlib
import joblib
from tqdm import tqdm

@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [ ]:
from joblib import Parallel, delayed

traits = ["uc", "ra", "cad", "ad", "scz"]
celltypes = ["PC3", "HT29", "HEK293T"]

combinations = []

for trait in traits:
    for celltype in celltypes:
        combinations.append((trait, celltype))

with tqdm_joblib(tqdm(desc="My calculation", total=len(combinations))) as progress_bar:
    Parallel(n_jobs=15)(delayed(full_knockdown)(trait, celltype) for (trait, celltype) in combinations)

# and once restrict perturbagens to those that are present in every cell type

In [ ]:
from joblib import Parallel, delayed

traits = ["uc", "ra", "cad", "ad", "scz"]
celltypes = ["PC3", "HT29", "HEK293T"]

combinations = []
restriction = set([column.split(".")[0] for column in pd.read_csv("/mnt/storage/cmap/2017/kd_df_HEK293T.tsv", header=0, sep="\t", index_col=0).columns.tolist()])
restriction = restriction.intersection(set([column.split(".")[0] for column in pd.read_csv("/mnt/storage/cmap/2017/kd_cgs_df_PC3.tsv", header=0, sep="\t", index_col=0).columns.tolist()]))
restriction = restriction.intersection(set([column.split(".")[0] for column in pd.read_csv("/mnt/storage/cmap/2017/kd_cgs_df_HT29.tsv", header=0, sep="\t", index_col=0).columns.tolist()]))


for trait in traits:
    for celltype in celltypes:
        combinations.append((trait, celltype))

with tqdm_joblib(tqdm(desc="My calculation", total=len(combinations))) as progress_bar:
    Parallel(n_jobs=15)(delayed(full_knockdown)(trait, celltype, id2hgnc.values(), list(restriction)) for trait, celltype in combinations)

# across traits (uncomment last lines to write file)

In [ ]:
import seaborn as sns
from random import sample
import numpy as np
import pandas as pd
from speos.visualization.settings import *
import matplotlib.pyplot as plt
from scipy.stats import rankdata

traits = ['uc', 'ra', 'cad', 'ad', 'scz']
celltypes = ["PC3", "HT29", "HEK293T"]

sign_perturbagens = {trait: {} for trait in traits}
background = {trait: {} for trait in traits}

n_random_draws = 5

for trait in traits:
    for celltype in celltypes:
        df = pd.read_csv("/mnt/storage/cmap/2017/differential_perturbation_knockdown_{}_{}.tsv".format(trait, celltype), sep="\t", header=0, index_col=0)
        sign_perturbagens[trait][celltype] = set(df.index[df["FDR"] < 0.05])
        background[trait][celltype] = set(df.index)

overlap_background = {trait: {} for trait in traits}
overlap_indices = {trait: {} for trait in traits}
rand_control = {trait: {} for trait in traits}
for traitA in traits:
    overlap_indices[traitA] = {trait: {} for trait in traits}
    overlap_background[traitA] = {trait: {} for trait in traits}
    rand_control[traitA] = {trait: {} for trait in traits}
    for traitB in traits:
        #union = sign_perturbagens[trait][celltypes[0]].union(sign_perturbagens[trait][celltypes[1]]).union(sign_perturbagens[trait][celltypes[2]])
        for celltypeA in celltypes:
            row = []
            background_row = []
            rand_row = []
            for celltypeB in celltypes:
                setA = sign_perturbagens[traitA][celltypeA]
                setB = sign_perturbagens[traitB][celltypeB]
                real_coeff = len(setA.intersection(setB)) / min(len(setA), len(setB))
                row.append(real_coeff)

                setA = background[traitA][celltypeA]
                setB = background[traitB][celltypeB]
                background_row.append(len(setA.intersection(setB)) / min(len(setA), len(setB)))

                rand_coeffs = []
                for _ in range(n_random_draws):
                    sampleA = sample(list(background[traitA][celltypeA]), len(sign_perturbagens[traitA][celltypeA]))
                    sampleB = sample(list(background[traitB][celltypeB]), len(sign_perturbagens[traitB][celltypeB]))
                    rand_coeffs.append(len(set(sampleA).intersection(set(sampleB))) / min(len(sampleA), len(sampleB)))
                
                #rand_coeffs.append(real_coeff)
                #rand_row.append((np.argpartition(rand_coeffs, n_random_draws) == n_random_draws).nonzero()[0].item())
                rand_row.append(rankdata(rand_coeffs + [real_coeff], "average")[-1].item())
            
            rand_control[traitA][traitB][celltypeA] = rand_row
            overlap_indices[traitA][traitB][celltypeA] = row
            overlap_background[traitA][traitB][celltypeA] = background_row

rownames = []
rows = []
for traitA in traits:
    for celltype in celltypes:
        rownames.append(celltype)
        row = []
        for traitB in traits:
            row.extend(overlap_indices[traitA][traitB][celltype])
        rows.append(row)

rows = np.asarray(rows)

rownames = []
background_rows = []
for traitA in traits:
    for celltype in celltypes:
        rownames.append(celltype)
        row = []
        for traitB in traits:
            row.extend(overlap_background[traitA][traitB][celltype])
        background_rows.append(row)

background_rows = np.asarray(background_rows)

rows = rows / background_rows

rows = rows[:,[0,3,6,9,12,1,4,7,10,13,2,5,8,11,14]]
rows = rows[[0,3,6,9,12,1,4,7,10,13,2,5,8,11,14], :]
    
from statsmodels.stats.multitest import fdrcorrection
import matplotlib as mpl

rand_rownames = []
rand_rows = []
for traitA in traits:
    for celltype in celltypes:
        rand_rownames.append(celltype)
        row = []
        for traitB in traits:
            row.extend(rand_control[traitA][traitB][celltype])
        rand_rows.append(row)

rand_rows = np.asarray(rand_rows)
rand_rows = rand_rows[:,[0,3,6,9,12,1,4,7,10,13,2,5,8,11,14]]
rand_rows = rand_rows[[0,3,6,9,12,1,4,7,10,13,2,5,8,11,14], :]

rand_rows =  rand_rows / (n_random_draws + 1)

labels = rand_rows.copy()
print(rand_rows)
labels[labels > 0.5] = 1 - labels[labels > 0.5]

old_shape = labels.shape

fdr_labels = fdrcorrection(labels.flatten())[1].reshape(old_shape)

rand_rows[rand_rows < 0.5] = fdr_labels[rand_rows < 0.5]
rand_rows[rand_rows > 0.5] = 1- fdr_labels[rand_rows > 0.5]

fdr_labels *= 2

for i in range(fdr_labels.shape[0]):
    for j in range(fdr_labels.shape[1]):
        if j <= i: 
            continue
        else:
            value = np.mean([fdr_labels[i,j], fdr_labels[j,i]])
            fdr_labels[i,j] = value
            fdr_labels[j,i] = value



#df = pd.DataFrame(rows, columns = [[celltype + "_" + trait.upper()  for celltype in celltypes for trait in traits]], index = [[celltype + "_" + trait.upper()  for celltype in celltypes for trait in traits]])
#df.to_csv("all_5_knockdown_overlap_new.tsv", sep="\t")
#df_fdr = pd.DataFrame(fdr_labels, columns = [[celltype + "_" + trait.upper()  for celltype in celltypes for trait in traits]], index = [[celltype + "_" + trait.upper()  for celltype in celltypes for trait in traits]])
#df_fdr.to_csv("all_5_knockdown_fdr_new.tsv", sep="\t")


In [ ]:
rows = pd.read_csv("all_5_knockdown_overlap.tsv", sep="\t", header=0, index_col=0).values
fdr_labels = pd.read_csv("all_5_knockdown_fdr.tsv", sep="\t", header=0, index_col=0).values

oldshape = rows.shape 
plot_labels = rows.flatten()

plot_labels = np.asarray([("%.2g" % k).lstrip('0') if k not in [1, 0] else k for k in plot_labels]).reshape(oldshape)

fig, ax = plt.subplots(figsize=(full_width*0.9*cm,full_width*0.8*cm ))

ax = sns.heatmap(rows, vmin=0.3,  vmax=1, cmap="Purples", annot=plot_labels, ax=ax, fmt="", annot_kws={"fontsize": 8}, zorder=1)
ax.set_yticklabels([trait.upper() for trait in traits]*3, rotation=90 )
ax.set_xticklabels([trait.upper() for trait in traits]*3, rotation=0, ha="center")
plt.yticks(rotation=0)

maximum = 15
minimum = 0
stride = 5
for trait, start in zip(rownames, range(minimum, maximum, stride)):
    ax.text(x = start + (stride/2), y= 17, s=trait, ha="center")
    ax.text(y = start + (stride/2), x= -2.5, s=trait, va="center", rotation=90)

cmap = mpl.colors.ListedColormap([(0.9, 0.9, 0.9, 1), (0,0,0,0)])

# create a normalize object the describes the limits of
# each color
bounds = [0., 0.5, 1.]
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
plot_labels[fdr_labels < 0.05] = ""

ax.imshow((fdr_labels < 0.5).astype(np.uint8), interpolation='none', cmap=cmap, norm=norm)
sns.heatmap((fdr_labels < 0.05).astype(np.uint8), vmin=0,  vmax=1, cmap=cmap, norm=norm, annot=plot_labels, ax=ax, fmt="", annot_kws={"fontsize": 8}, cbar=False, zorder=2)

ax.set_yticklabels([trait.upper() + "\n$\\regular_{n=%s}$" % len(sign_perturbagens[trait][celltype]) for celltype in celltypes for trait in traits], rotation=90 )
ax.set_xticklabels([trait.upper() + "\n$\\regular_{n=%s}$" % len(sign_perturbagens[trait][celltype]) for celltype in celltypes for trait in traits], rotation=0, ha="center")
plt.yticks(rotation=0)

plt.savefig("across_traits_knockdown_5.svg", bbox_inches="tight")

In [ ]:
import seaborn as sns
from random import sample
import numpy as np
import pandas as pd
from speos.visualization.settings import *
import matplotlib.pyplot as plt
from scipy.stats import rankdata

traits = ['uc', 'ra', 'cad', 'ad', 'scz']
celltypes = ["PC3", "HT29", "HEK293T"]

sign_perturbagens = {trait: {} for trait in traits}
background = {trait: {} for trait in traits}

n_random_draws = 1

for trait in traits:
    for celltype in celltypes:
        df = pd.read_csv("/mnt/storage/cmap/2017/differential_perturbation_overexpression_{}_{}.tsv".format(trait, celltype), sep="\t", header=0, index_col=0)
        sign_perturbagens[trait][celltype] = set(df.index[df["FDR"] < 0.05])
        background[trait][celltype] = set(df.index)

overlap_background = {trait: {} for trait in traits}
overlap_indices = {trait: {} for trait in traits}
rand_control = {trait: {} for trait in traits}
for traitA in traits:
    overlap_indices[traitA] = {trait: {} for trait in traits}
    overlap_background[traitA] = {trait: {} for trait in traits}
    rand_control[traitA] = {trait: {} for trait in traits}
    for traitB in traits:
        #union = sign_perturbagens[trait][celltypes[0]].union(sign_perturbagens[trait][celltypes[1]]).union(sign_perturbagens[trait][celltypes[2]])
        for celltypeA in celltypes:
            row = []
            background_row = []
            rand_row = []
            for celltypeB in celltypes:
                setA = sign_perturbagens[traitA][celltypeA]
                setB = sign_perturbagens[traitB][celltypeB]
                real_coeff = len(setA.intersection(setB)) / min(len(setA), len(setB))
                row.append(real_coeff)

                setA = background[traitA][celltypeA]
                setB = background[traitB][celltypeB]
                background_row.append(len(setA.intersection(setB)) / min(len(setA), len(setB)))

                rand_coeffs = []
                for _ in range(n_random_draws):
                    sampleA = sample(list(background[traitA][celltypeA]), len(sign_perturbagens[traitA][celltypeA]))
                    sampleB = sample(list(background[traitB][celltypeB]), len(sign_perturbagens[traitB][celltypeB]))
                    rand_coeffs.append(len(set(sampleA).intersection(set(sampleB))) / min(len(sampleA), len(sampleB)))
                
                #rand_coeffs.append(real_coeff)
                #rand_row.append((np.argpartition(rand_coeffs, n_random_draws) == n_random_draws).nonzero()[0].item())
                rand_row.append(rankdata(rand_coeffs + [real_coeff], "average")[-1].item())
            
            rand_control[traitA][traitB][celltypeA] = rand_row
            overlap_indices[traitA][traitB][celltypeA] = row
            overlap_background[traitA][traitB][celltypeA] = background_row

rownames = []
rows = []
for traitA in traits:
    for celltype in celltypes:
        rownames.append(celltype)
        row = []
        for traitB in traits:
            row.extend(overlap_indices[traitA][traitB][celltype])
        rows.append(row)

rows = np.asarray(rows)

rownames = []
background_rows = []
for traitA in traits:
    for celltype in celltypes:
        rownames.append(celltype)
        row = []
        for traitB in traits:
            row.extend(overlap_background[traitA][traitB][celltype])
        background_rows.append(row)

background_rows = np.asarray(background_rows)

rows = rows / background_rows

rows = rows[:,[0,3,6,9,12,1,4,7,10,13,2,5,8,11,14]]
rows = rows[[0,3,6,9,12,1,4,7,10,13,2,5,8,11,14], :]



#plt.savefig("across_traits_knockdown_5_restricted.svg", bbox_inches="tight")
    
from statsmodels.stats.multitest import fdrcorrection
import matplotlib as mpl

rand_rownames = []
rand_rows = []
for traitA in traits:
    for celltype in celltypes:
        rand_rownames.append(celltype)
        row = []
        for traitB in traits:
            row.extend(rand_control[traitA][traitB][celltype])
        rand_rows.append(row)

rand_rows = np.asarray(rand_rows)
rand_rows = rand_rows[:,[0,3,6,9,12,1,4,7,10,13,2,5,8,11,14]]
rand_rows = rand_rows[[0,3,6,9,12,1,4,7,10,13,2,5,8,11,14], :]

rand_rows =  rand_rows / (n_random_draws + 1)

labels = rand_rows.copy()
labels[labels > 0.5] = 1 - labels[labels > 0.5]

old_shape = labels.shape

fdr_labels = fdrcorrection(labels.flatten())[1].reshape(old_shape)

fdr_labels *= 2

for i in range(fdr_labels.shape[0]):
    for j in range(fdr_labels.shape[1]):
        if j <= i: 
            continue
        else:
            value = np.mean([fdr_labels[i,j], fdr_labels[j,i]])
            fdr_labels[i,j] = value
            fdr_labels[j,i] = value


#df = pd.DataFrame(rows, columns = [[celltype + "_" + trait.upper()  for celltype in celltypes for trait in traits]], index = [[celltype + "_" + trait.upper()  for celltype in celltypes for trait in traits]])
#df.to_csv("all_5_overexpression_overlap.tsv", sep="\t")
#df_fdr = pd.DataFrame(fdr_labels, columns = [[celltype + "_" + trait.upper()  for celltype in celltypes for trait in traits]], index = [[celltype + "_" + trait.upper()  for celltype in celltypes for trait in traits]])
#df_fdr.to_csv("all_5_overexpression_fdr.tsv", sep="\t")

In [ ]:
import seaborn as sns
from random import sample
import numpy as np
import pandas as pd
from speos.visualization.settings import *
import matplotlib.pyplot as plt

rows = pd.read_csv("all_5_overexpression_overlap.tsv", sep="\t", header=0, index_col=0).values
fdr_labels = pd.read_csv("all_5_overexpression_fdr.tsv", sep="\t", header=0, index_col=0).values

oldshape = rows.shape 
plot_labels = rows.flatten()

plot_labels = np.asarray([("%.2g" % k).lstrip('0') if k not in [1, 0] else k for k in plot_labels]).reshape(oldshape)

fig, ax = plt.subplots(figsize=(full_width*0.9*cm,full_width*0.8*cm ))

ax = sns.heatmap(rows, vmin=0.3,  vmax=1, cmap="Oranges", annot=plot_labels, ax=ax, fmt="", annot_kws={"fontsize": 8}, zorder=1)
ax.set_yticklabels([trait.upper() for trait in traits]*3, rotation=90 )
ax.set_xticklabels([trait.upper() for trait in traits]*3, rotation=0, ha="center")
plt.yticks(rotation=0)

maximum = 15
minimum = 0
stride = 5
for trait, start in zip(rownames, range(minimum, maximum, stride)):
    ax.text(x = start + (stride/2), y= 17, s=trait, ha="center")
    ax.text(y = start + (stride/2), x= -2.5, s=trait, va="center", rotation=90)

cmap = mpl.colors.ListedColormap([(0.9, 0.9, 0.9, 1), (0,0,0,0)])

# create a normalize object the describes the limits of
# each color
bounds = [0., 0.5, 1.]
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
plot_labels[fdr_labels < 0.05] = ""

ax.imshow((fdr_labels < 0.5).astype(np.uint8), interpolation='none', cmap=cmap, norm=norm)
sns.heatmap((fdr_labels < 0.05).astype(np.uint8), vmin=0,  vmax=1, cmap=cmap, norm=norm, annot=plot_labels, ax=ax, fmt="", annot_kws={"fontsize": 8}, cbar=False, zorder=2)
ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False, right=True, left=False, labelright=True, labelleft=False)
ax.set_yticklabels([trait.upper() + "\n$\\regular_{n=%s}$" % len(sign_perturbagens[trait][celltype]) for celltype in celltypes for trait in traits], rotation=90 )
ax.set_xticklabels([trait.upper() + "\n$\\regular_{n=%s}$" % len(sign_perturbagens[trait][celltype]) for celltype in celltypes for trait in traits], rotation=0, ha="center")
plt.yticks(rotation=0)

maximum = 15
minimum = 0
stride = 5
#or trait, start in zip(rownames, range(minimum, maximum, stride)):
#   ax.text(x = start + (stride/2), y= 15, s=trait, ha="center")
#   ax.text(y = start + (stride/2), x= -2, s=trait, va="center", rotation=90)

plt.savefig("across_traits_overexpression_5_new.svg", bbox_inches="tight")

In [ ]:
import seaborn as sns
from random import sample
import numpy as np
import pandas as pd
from speos.visualization.settings import *
import matplotlib.pyplot as plt
from scipy.stats import rankdata

traits = ['uc', 'ra', 'cad', 'ad', 'scz']
celltypes = ["PC3", "HT29", "HEK293T"]


fig, axs = plt.subplots(ncols= 3, figsize=(full_width*cm,full_width*0.25*cm )) 

allrows = pd.read_csv("all_5_knockdown_overlap.tsv", sep="\t", header=0, index_col=0)
allfdr = pd.read_csv("all_5_knockdown_fdr.tsv", sep="\t", header=0, index_col=0)

for ax, celltype in zip(axs, celltypes):

    colnames = [colname for colname in allrows.columns if colname.startswith(celltype)]
    rows = allrows.loc[colnames, colnames].values
    oldshape = rows.shape 
    plot_labels = rows.flatten()

    plot_labels = np.asarray([("%.2g" % k).lstrip('0') if k not in [1, 0] else k for k in plot_labels]).reshape(oldshape)

    ax = sns.heatmap(rows, vmin=0.3,  vmax=1, cmap="Purples", annot=plot_labels, ax=ax, fmt="", annot_kws={"fontsize": 8}, zorder=1)

    fdr_labels = allfdr.loc[colnames, colnames].values
    cmap = mpl.colors.ListedColormap([(0.8, 0.8, 0.8, 1), (0,0,0,0)])

    # create a normalize object the describes the limits of
    # each color
    bounds = [0., 0.5, 1.]
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    plot_labels[fdr_labels < 0.05] = ""

    #ax.imshow((fdr_labels < 0.5).astype(np.uint8), interpolation='none', cmap=cmap, norm=norm)
    sns.heatmap((fdr_labels < 0.05).astype(np.uint8), vmin=0,  vmax=1, cmap=cmap, norm=norm, annot=plot_labels, ax=ax, fmt="", annot_kws={"fontsize": 8}, cbar=False, zorder=2)

    ax.set_yticklabels([trait.upper() + "\n$\\regular_{n=%s}$" % len(sign_perturbagens[trait][celltype]) for trait in traits], rotation=90 )
    ax.set_xticklabels([trait.upper() + "\n$\\regular_{n=%s}$" % len(sign_perturbagens[trait][celltype]) for trait in traits], rotation=0, ha="center")
    ax.tick_params(axis='y', labelrotation=0)
    ax.set_title(celltype)

        

plt.savefig("across_traits_knockdown.svg".format(celltype), bbox_inches="tight")

In [ ]:
import seaborn as sns
from random import sample
import numpy as np
import pandas as pd
from speos.visualization.settings import *
import matplotlib.pyplot as plt
from scipy.stats import rankdata

fig, axs = plt.subplots(ncols= 3, figsize=(full_width*cm,full_width*0.25*cm )) 

allrows = pd.read_csv("all_5_overexpression_overlap.tsv", sep="\t", header=0, index_col=0)
allfdr = pd.read_csv("all_5_overexpression_fdr.tsv", sep="\t", header=0, index_col=0)

traits = ['uc', 'ra', 'cad', 'ad', 'scz']
celltypes = ["PC3", "HT29", "HEK293T"]
for ax, celltype in zip(axs, celltypes):

    colnames = [colname for colname in allrows.columns if colname.startswith(celltype)]
    rows = allrows.loc[colnames, colnames].values
    oldshape = rows.shape 
    plot_labels = rows.flatten()

    plot_labels = np.asarray([("%.2g" % k).lstrip('0') if k not in [0, 1] else k for k in plot_labels]).reshape(oldshape)

    

    ax = sns.heatmap(rows, vmin=0.3,  vmax=1, cmap="Oranges", annot=plot_labels, ax=ax, fmt="", annot_kws={"fontsize": 8}, zorder=1)

    fdr_labels = allfdr.loc[colnames, colnames].values
    cmap = mpl.colors.ListedColormap([(0.8, 0.8, 0.8, 1), (0,0,0,0)])

    # create a normalize object the describes the limits of
    # each color
    bounds = [0., 0.5, 1.]
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    plot_labels[fdr_labels < 0.05] = ""

    #ax.imshow((fdr_labels < 0.5).astype(np.uint8), interpolation='none', cmap=cmap, norm=norm)
    sns.heatmap((fdr_labels < 0.05).astype(np.uint8), vmin=0,  vmax=1, cmap=cmap, norm=norm, annot=plot_labels, ax=ax, fmt="", annot_kws={"fontsize": 8}, cbar=False, zorder=2)

    ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False, right=True, left=False, labelright=True, labelleft=False)
    ax.set_yticklabels([trait.upper() + "\n$\\regular_{n=%s}$" % len(sign_perturbagens[trait][celltype]) for trait in traits], rotation=90 )
    ax.set_xticklabels([trait.upper() + "\n$\\regular_{n=%s}$" % len(sign_perturbagens[trait][celltype]) for trait in traits], rotation=0, ha="center")
    ax.tick_params(axis='y', labelrotation=0)
    ax.set_title(celltype)
        

plt.savefig("across_traits_overexpression.svg", bbox_inches="tight")

# Volcano plot

In [ ]:
trait = "uc"
celltype = "HT29"

allcore, other_coregenes, hsps,  noncore = get_coregenes(trait, id2hgnc.values())
full_kd_df = pd.read_csv("/mnt/storage/cmap/2017/kd_cgs_df_{}.tsv".format(celltype), header=0, sep="\t", index_col=0)

gene_df = pd.read_csv("/mnt/storage/cmap/2017/GSE92742_Broad_LINCS_gene_info.txt", sep="\t", header=0, index_col=None)
good_genes = gene_df.pr_gene_symbol[gene_df.pr_is_bing == 1].tolist()

full_kd_df = full_kd_df.loc[full_kd_df.index.isin(good_genes), :]

results_df, total, core_result, hsp_result, peri_result = get_differential_percentages(full_kd_df, allcore, hsps, noncore, use_mean=True)
results_df = results_df.transpose()
print(len(results_df))
results_df = results_df[(results_df["Core Gene"] + results_df["HSP"] + results_df["Peripheral"]).values.astype(np.bool_)]
print(len(results_df))
results_df = results_df[results_df["FDR"] < 0.05]
print(len(results_df))

In [ ]:
from speos.visualization.settings import *
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
from adjustText import adjust_text

fig, ax = plt.subplots(figsize=(full_width*cm*0.5, 8*cm))
all_significant = results_df["meandiff"][(results_df["FDR"] < 0.05) & (results_df["Core Gene"] + results_df["HSP"] + results_df["Peripheral"])]
core_significant = results_df["meandiff"][(results_df["FDR"] < 0.05) & (results_df["Core Gene"])]
hsp_significant = results_df["meandiff"][(results_df["FDR"] < 0.05) & (results_df["HSP"])]
peri_significant = results_df["meandiff"][(results_df["FDR"] < 0.05) & (results_df["Peripheral"])]

core_fdr = results_df["FDR"][(results_df["FDR"] < 0.05) & (results_df["Core Gene"])]
hsp_fdr = results_df["FDR"][(results_df["FDR"] < 0.05) & (results_df["HSP"])]
peri_fdr = results_df["FDR"][(results_df["FDR"] < 0.05) & (results_df["Peripheral"])]
ax.set_yscale("log")
ax.scatter(x=peri_significant,y = 1 / peri_fdr, s=5, c="#8a8a8a")
ax.scatter(x=core_significant,y = 1 / core_fdr, s=5, c="#01016f")
ax.scatter(x=hsp_significant,y = 1 / hsp_fdr, s=5, c="#d8031c")

texts = []
sorted_df = results_df.sort_values(by="FDR", ascending=True)
already_printed = []
for i in range(8):
    texts.append(ax.text(sorted_df["meandiff"][i], 1/sorted_df["FDR"][i], sorted_df.index[i], size=4, va="center"))
    already_printed.append(sorted_df.index[i])

adjust_text(texts, x=results_df["meandiff"].values.tolist(), y= (1/results_df["FDR"].values).tolist(), force_points=3, arrowprops=dict(arrowstyle='-', color='black', lw=0.5), ax=ax)

texts = []
sorted_df = results_df.sort_values(by="meandiff", ascending=True)
for i in range(3):
    if sorted_df.index[i] not in already_printed:
      texts.append(ax.text(sorted_df["meandiff"][i], 1/sorted_df["FDR"][i], sorted_df.index[i], size=4, va="center"))
      already_printed.append(sorted_df.index[i])

sorted_df = results_df.sort_values(by="meandiff", ascending=False)
for i in range(5):
    if sorted_df.index[i] not in already_printed:
      texts.append(ax.text(sorted_df["meandiff"][i], 1/sorted_df["FDR"][i], sorted_df.index[i], size=4, va="center"))
      already_printed.append(sorted_df.index[i])

sorted_df = results_df[results_df["meandiff"] < 0].sort_values(by="FDR", ascending=True)
for i in range(5):
    if sorted_df.index[i] not in already_printed:
      texts.append(ax.text(sorted_df["meandiff"][i], 1/sorted_df["FDR"][i], sorted_df.index[i], size=4, va="center"))
      already_printed.append(sorted_df.index[i])

adjust_text(texts, x=results_df["meandiff"].values.tolist(), y=(1/results_df["FDR"].values).tolist(), force_points=5, arrowprops=dict(arrowstyle='-', color='black', lw=0.5), ax=ax)

texts = []
hsp_df = results_df[results_df["HSP"]]
if len(hsp_df) > 0:
      for i in range(len(hsp_df)):
            texts.append(ax.text(hsp_df["meandiff"][i], 1/hsp_df["FDR"][i], hsp_df.index[i], size=4, va="center"))

      adjust_text(texts, x=results_df["meandiff"].values.tolist(), y=(1/results_df["FDR"].values).tolist(), force_points=0.5, arrowprops=dict(arrowstyle='-', color='black', lw=0.5), ax=ax)


ax.vlines(0, 1/0.05, 10e28, color="gray", linestyles=":")

ax.text(-0.01, y=10e28, s="{:.1f}%".format(((all_significant < 0).sum() / len(all_significant)) * 100), ha="right", va="top", fontsize=8)
ax.text(+0.01, y=10e28, s="{:.1f}%".format(((all_significant > 0).sum() / len(all_significant)) * 100), ha="left", va="top", fontsize=8)


ax.text(-0.01, y=10e26, s="{:.1f}%".format(((peri_significant < 0).sum() / len(peri_significant)) * 100), ha="right", va="top", color="#8a8a8a", fontsize=8)
ax.text(+0.01, y=10e26, s="{:.1f}%".format(((peri_significant > 0).sum() / len(peri_significant)) * 100), ha="left", va="top", color="#8a8a8a", fontsize=8)

ax.text(-0.01, y=10e24, s="{:.1f}%".format(((core_significant < 0).sum() / len(core_significant)) * 100), ha="right", va="top", color="#01016f", fontsize=8)
ax.text(+0.01, y=10e24, s="{:.1f}%".format(((core_significant > 0).sum() / len(core_significant)) * 100), ha="left", va="top", color="#01016f", fontsize=8)

ax.text(-0.01, y=10e22, s="{:.1f}%".format(((hsp_significant < 0).sum() / len(hsp_significant)) * 100), ha="right", va="top", color="#d8031c", fontsize=8)
ax.text(+0.01, y=10e22, s="{:.1f}%".format(((hsp_significant > 0).sum() / len(hsp_significant)) * 100), ha="left", va="top", color="#d8031c", fontsize=8)


legend_elements = [Patch(facecolor='black', edgecolor='black',
                         label='Any\nn={}'.format(len(all_significant))),
                   Patch(facecolor='#8a8a8a', edgecolor='#8a8a8a',
                         label='Peripheral\nn={}'.format(len(peri_significant))),
                   Patch(facecolor='#01016f', edgecolor='#01016f',
                         label='Core Gene\nn={}'.format(len(core_significant))),
                   Patch(facecolor='#d8031c', edgecolor="#d8031c",
                         label='HSP\nn={}'.format(len(hsp_significant)))]


leg = ax.legend(handles=legend_elements, loc='upper left', title="Perturbagen", fontsize=6.8, title_fontsize=7, ncol=2, columnspacing=1.7, handletextpad=-0.7)

for patch in leg.get_patches():
    patch.set_height(15)
    patch.set_width(5)
    patch.set_y(-5)

ax.set_ylim(bottom=5, top=10e42)
ax.set_ylabel(r"$-\log(FDR)$")
ax.set_xlabel("Mean Differential Perturbation\n(Core Gene - Peripheral)")
plt.tight_layout()
plt.savefig("Volcano_Knockdown_strongest_{}_{}.svg".format(trait, celltype), bbox_inches="tight")


# Now for Overexpression

In [ ]:
celltype = "HT29"

full_oe_df = pd.read_csv("/mnt/storage/cmap/2017/oe_df_{}.tsv".format(celltype), header=0, sep="\t", index_col=0)
gene_df = pd.read_csv("/mnt/storage/cmap/2017/GSE92742_Broad_LINCS_gene_info.txt", sep="\t", header=0, index_col=None)
good_genes = gene_df.pr_gene_symbol[gene_df.pr_is_bing == 1].tolist()

full_oe_df = full_oe_df.loc[full_oe_df.index.isin(good_genes), :]

In [ ]:
get_differential_percentages(full_oe_df, allcore, hsps, noncore)[0].transpose().to_csv("/mnt/storage/cmap/2017/differential_perturbation_overexpression_UC_{}.tsv".format(celltype), sep="\t")

In [ ]:
results_df, total, core_result, hsp_result, peri_result = get_differential_percentages(full_oe_df, allcore, hsps, noncore)


random_core = []
random_hsp = []
random_peri = []
for i in tqdm(range(2)):
    _, _, core_result_random, hsp_result_random, peri_result_random = get_differential_percentages(full_oe_df, allcore, hsps, noncore, randomize_core=True, random_seed=i)
    random_core.append(core_result_random[0])
    random_hsp.append(hsp_result_random[0])
    random_peri.append(peri_result_random[0])


In [ ]:
import seaborn as sns
from speos.visualization.settings import *
import matplotlib.pyplot as plt
fig, ax= plt.subplots(figsize=(8*cm,5*cm))

num_target_core_genes = len(allcore.intersection(set(full_kd_df.index)))

kd_matrix_mean = pd.DataFrame(index=["HSP\n(n={})".format(hsp_result[1]), "Peripheral\n(n={})".format(peri_result[1]), "Core\n(n={})".format(core_result[1])],
                         data={"Core Genes\nn={}".format(num_target_core_genes): [hsp_result[0], peri_result[0],  core_result[0]],
                               "Random Genes\nn={} ({}x)".format(num_target_core_genes, len(random_hsp)): [np.mean(random_hsp), np.mean(random_peri), np.mean(random_core)]})

ax = sns.heatmap(kd_matrix_mean.transpose(),vmin=0, vmax=1, cmap="Oranges", annot=True, fmt=".1%", ax=ax,
                 cbar_kws={'label': "Fraction Significant\nDifferential Perturbations",
                           "pad": 0.01})
cbar = ax.collections[-1].colorbar
cbar.ax.set_ylabel("Fraction Significant\nDifferential Perturbations", fontsize=5)
ax.set_ylabel("Target Gene Set", fontsize=7)
ax.set_xlabel("Perturbagen (Overexpression)", fontsize=7)
plt.tight_layout()

#plt.savefig("Perurbation_overexpression_{}.svg".format(celltype), bbox_inches="tight")

In [ ]:

def full_overexpression(trait, celltype, restriction=set(), nrandom=100):
    import matplotlib as mpl
    from speos.utils.config import Config
    from speos.preprocessing.handler import InputHandler
    import os 
    import pandas as pd
    import seaborn as sns
    
    # set font
    mpl.rcParams['font.family'] = 'Helvetica'

    full_width = 18
    cm = 1/2.54
    small_font = 6
    medium_font = 8
    large_font = 10
    mpl.rc('xtick', labelsize=small_font)
    mpl.rc('ytick', labelsize=small_font)
    mpl.rcParams['axes.linewidth'] = 0.4
    mpl.rcParams['ytick.major.size'] = 3
    mpl.rcParams['ytick.major.width'] = 0.5
    mpl.rcParams['ytick.minor.size'] = 2
    mpl.rcParams['ytick.minor.width'] = 0.3
    mpl.rcParams['xtick.major.size'] = 2
    mpl.rcParams['xtick.major.width'] = 0.3
    mpl.rcParams['xtick.minor.size'] = 1
    mpl.rcParams['xtick.minor.width'] = 0.1

    os.chdir("..")
    config = Config()
    if trait == "ad":
        configstring = "alz"
    elif trait == "cad":
        configstring = "cad_really"
    else:
        configstring = trait
    config.parse_yaml("/home/ubuntu/speos/config_{}_only_nohetio_film_newstorage.yaml".format(configstring))
    prepro = InputHandler(config).get_preprocessor()
    prepro.build_graph(adjacency=False)
    background = set(prepro.id2hgnc.values())
    os.chdir("notebooks")

    print ("Starting OE Analysis for {} {}".format(trait, celltype))
    if isinstance(trait, str):
        allcore, other_coregenes, hsps,  noncore = get_coregenes(trait, background)
        traitstring = trait
    else:
        allcore = set()
        other_coregenes = set()
        hsps = set()
        noncore = set(list(background)[:])
        for _trait in trait:
            _allcore, _other_coregenes, _hsps,  _noncore = get_coregenes(_trait, background)
            allcore.update(set(_allcore))
            other_coregenes.update(set(_other_coregenes))
            hsps.update(set(_hsps))
            noncore = noncore.intersection(_noncore)
        traitstring = "_".join(trait)



    full_df = pd.read_csv("/mnt/storage/cmap/2017/oe_df_{}.tsv".format(celltype), header=0, sep="\t", index_col=0)

    gene_df = pd.read_csv("/mnt/storage/cmap/2017/GSE92742_Broad_LINCS_gene_info.txt", sep="\t", header=0, index_col=None)
    good_genes = gene_df.pr_gene_symbol[gene_df.pr_is_bing == 1].tolist()

    full_df = full_df.loc[full_df.index.isin(good_genes), :]

    if len(restriction) > 0:
        restriction = [column for column in full_df.columns if column.split(".")[0] in restriction]
        full_df = full_df[list(restriction)]
        typestring = celltype + "_restricted"
    else:
        typestring = celltype
    
    get_differential_percentages(full_df, allcore, hsps, noncore, use_mean=True)[0].transpose().to_csv("/mnt/storage/cmap/2017/differential_perturbation_overexpression_{}_{}.tsv".format(traitstring, typestring), sep="\t")

    results_df, total, core_result, hsp_result, peri_result = get_differential_percentages(full_df, allcore, hsps, noncore, use_mean=True)
    
    gwas_genes = set(pd.read_csv("../hsps/gwas_genes_closest/5e-8/uc_gwas_genes.tsv", header=0, sep="\t", index_col=None)["HGNC"].tolist()).intersection(prepro.id2hgnc.values())
    gwas_results_df, _, gwas_core_result, gwas_hsp_result, gwas_peri_result = get_differential_percentages(full_df, allcore, hsps, noncore,  use_min=False, use_mean=True, alternative_core=gwas_genes)
    gwas_results_df.transpose().to_csv("/mnt/storage/cmap/2017/differential_perturbation_overexpression_gwas_{}_{}.tsv".format(traitstring, typestring), sep="\t")


    random_core = []
    random_hsp = []
    random_peri = []
    for i in range(nrandom):
        _, _, core_result_random, hsp_result_random, peri_result_random = get_differential_percentages(full_df, allcore, hsps, noncore,use_min=False, use_mean=True, randomize_core=True, random_seed=i)
        random_core.append(core_result_random[0])
        random_hsp.append(hsp_result_random[0])
        random_peri.append(peri_result_random[0])

    results_df = results_df.transpose()
        
    fig, ax= plt.subplots(figsize=(8*cm,4*cm))

    num_target_core_genes = len(allcore.intersection(set(full_kd_df.index)))

    
    kd_matrix_mean = pd.DataFrame(index=["HSP" + "\n(n=%s)" % hsp_result[1], "Peripheral\n" + "(n=%s)" % peri_result[1], "Core Gene\n" + "(n=%s)" % core_result[1]],
                         data={"Core Genes\n" + "n={}".format(num_target_core_genes): [hsp_result[0], peri_result[0],  core_result[0]],
                               "GWAS Genes\n" + "n={}".format(len(gwas_genes)): [gwas_hsp_result[0], gwas_peri_result[0],  gwas_core_result[0]],
                               "Random Genes\n" + "n={} ({}x)".format(num_target_core_genes, len(random_hsp)): [np.mean(random_hsp), np.mean(random_peri), np.mean(random_core)]})

    ax = sns.heatmap(kd_matrix_mean.transpose(), vmin=0,  vmax=1, cmap="Oranges", annot=True, fmt=".1%", ax=ax, 
                 annot_kws = {"fontsize": 8},
                 cbar_kws={'label': "Fraction Significant\nDifferential Perturbations",
                           "pad": 0.01})
    ax.tick_params(axis='y', labelrotation=0)
    ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 6)
    cbar = ax.collections[-1].colorbar
    cbar.ax.set_ylabel("Fraction Discriminative\nPerturbations", fontsize=7)
    ax.set_ylabel("Target Gene Set", fontsize=7)
    ax.set_xlabel("Perturbagen (Overexpression)", fontsize=7)
    plt.tight_layout()
    plt.savefig("Perturbation_overexpression_{}_{}.svg".format(traitstring, typestring), bbox_inches="tight")

In [ ]:
full_overexpression("uc", "HT29", nrandom=100)

In [ ]:
from joblib import Parallel, delayed

traits = ["uc", "ra", "cad", "ad", "scz"]
celltypes = ["PC3", "HT29", "HEK293T"]

combinations = []

for trait in traits:
    for celltype in celltypes:
        combinations.append((trait, celltype))

with tqdm_joblib(tqdm(desc="My calculation", total=len(combinations))) as progress_bar:
    Parallel(n_jobs=len(combinations))(delayed(full_overexpression)(trait, celltype) for (trait, celltype) in combinations)

# and restricted

In [ ]:
from joblib import Parallel, delayed

traits = ["uc", "ra", "cad", "ad", "scz"]
celltypes = ["PC3", "HT29", "HEK293T"]

combinations = []
restriction = set(pd.read_csv("/mnt/storage/cmap/2017/oe_df_HEK293T.tsv", header=0, sep="\t", index_col=0).columns.tolist())
restriction = restriction.intersection(set(pd.read_csv("/mnt/storage/cmap/2017/oe_df_PC3.tsv", header=0, sep="\t", index_col=0).columns.tolist()))
restriction = restriction.intersection(set(pd.read_csv("/mnt/storage/cmap/2017/oe_df_HT29.tsv", header=0, sep="\t", index_col=0).columns.tolist()))



for trait in traits:
    for celltype in celltypes:
        combinations.append((trait, celltype))
        #restrictions.append(pd.read_csv("/mnt/storage/cmap/2017/oe_df_HEK293T.tsv", header=0, sep="\t", index_col=0).columns.tolist())

with tqdm_joblib(tqdm(desc="My calculation", total=len(combinations))) as progress_bar:
    Parallel(n_jobs=len(combinations))(delayed(full_overexpression)(trait, celltype, id2hgnc.values(), restriction) for trait, celltype in combinations)

In [ ]:
celltype = "HT29"

full_df = pd.read_csv("/mnt/storage/cmap/2017/oe_df_{}.tsv".format(celltype), header=0, sep="\t", index_col=0)

allcore, _, hsps, noncore = get_coregenes("uc", id2hgnc.values())

gene_df = pd.read_csv("/mnt/storage/cmap/2017/GSE92742_Broad_LINCS_gene_info.txt", sep="\t", header=0, index_col=None)
good_genes = gene_df.pr_gene_symbol[gene_df.pr_is_bing == 1].tolist()

full_df = full_df.loc[full_df.index.isin(good_genes), :]


results_df, total, core_result, hsp_result, peri_result = get_differential_percentages(full_df, allcore, hsps, noncore, use_min=True)
results_df = results_df.transpose()
results_df = results_df[(results_df["Core Gene"] + results_df["HSP"] + results_df["Peripheral"]).values.astype(np.bool_)]
results_df = results_df[results_df["FDR"] < 0.05]

In [ ]:
from speos.visualization.settings import *
from matplotlib.patches import Patch
from adjustText import adjust_text
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(full_width*cm*0.5, 8*cm))
ax.set_ylim(bottom=5, top=1e65)
ax.set_xlim(left=-0.7)
all_significant = results_df["meandiff"]
core_significant = results_df["meandiff"][results_df["Core Gene"]]
hsp_significant = results_df["meandiff"][results_df["HSP"]]
peri_significant = results_df["meandiff"][results_df["Peripheral"]]

core_fdr = results_df["FDR"][(results_df["FDR"] < 0.05) & (results_df["Core Gene"])]
hsp_fdr = results_df["FDR"][(results_df["FDR"] < 0.05) & (results_df["HSP"])]
peri_fdr = results_df["FDR"][(results_df["FDR"] < 0.05) & (results_df["Peripheral"])]
ax.set_yscale("log")
ax.scatter(x=peri_significant,y = 1 / peri_fdr, s=5, c="#8a8a8a")
ax.scatter(x=core_significant,y = 1 / core_fdr, s=5, c="#01016f")
ax.scatter(x=hsp_significant,y = 1 / hsp_fdr, s=5, c="#d8031c")


texts = []
sorted_df = results_df.sort_values(by="FDR", ascending=True)
already_printed = []
for i in range(8):
    texts.append(ax.text(sorted_df["meandiff"][i], 1/sorted_df["FDR"][i], sorted_df.index[i], size=4, va="center"))
    already_printed.append(sorted_df.index[i])

adjust_text(texts, x=results_df["meandiff"].values.tolist(), y= (1/results_df["FDR"].values).tolist(), force_points=3, arrowprops=dict(arrowstyle='-', color='black', lw=0.5), ax=ax)

texts = []
sorted_df = results_df.sort_values(by="meandiff", ascending=True)
for i in range(3):
    if sorted_df.index[i] not in already_printed:
      texts.append(ax.text(sorted_df["meandiff"][i], 1/sorted_df["FDR"][i], sorted_df.index[i], size=4, va="center"))
      already_printed.append(sorted_df.index[i])

sorted_df = results_df.sort_values(by="meandiff", ascending=False)
for i in range(8):
    if sorted_df.index[i] not in already_printed:
      texts.append(ax.text(sorted_df["meandiff"][i], 1/sorted_df["FDR"][i], sorted_df.index[i], size=4, va="center"))
      already_printed.append(sorted_df.index[i])

sorted_df = results_df[results_df["meandiff"] < 0].sort_values(by="FDR", ascending=True)
for i in range(3):
    if sorted_df.index[i] not in already_printed:
      texts.append(ax.text(sorted_df["meandiff"][i], 1/sorted_df["FDR"][i], sorted_df.index[i], size=4, va="center"))
      already_printed.append(sorted_df.index[i])

adjust_text(texts, x=results_df["meandiff"].values.tolist(), y=(1/results_df["FDR"].values).tolist(), force_points=10, arrowprops=dict(arrowstyle='-', color='black', lw=0.5), ax=ax)


texts = []
hsp_df = results_df[results_df["HSP"]]
for i in range(len(hsp_df)):
   texts.append(ax.text(hsp_df["meandiff"][i], 1/hsp_df["FDR"][i], hsp_df.index[i], size=4, va="center"))

adjust_text(texts, x=results_df["meandiff"].values.tolist(), y=(1/results_df["FDR"].values).tolist(), force_points=0.5, arrowprops=dict(arrowstyle='-', color='black', lw=0.5), ax=ax)


ax.vlines(0, 1/0.05, 10e40, color="gray", linestyles=":")

ax.text(-0.01, y=10e39, s="{:.1f}%".format(((all_significant < 0).sum() / len(all_significant)) * 100), ha="right", va="top", fontsize=8)
ax.text(+0.01, y=10e39, s="{:.1f}%".format(((all_significant > 0).sum() / len(all_significant)) * 100), ha="left", va="top", fontsize=8)

ax.text(-0.01, y=10e36, s="{:.1f}%".format(((peri_significant < 0).sum() / len(peri_significant)) * 100), ha="right", va="top", color="#8a8a8a", fontsize=8)
ax.text(+0.01, y=10e36, s="{:.1f}%".format(((peri_significant > 0).sum() / len(peri_significant)) * 100), ha="left", va="top", color="#8a8a8a", fontsize=8)

ax.text(-0.01, y=10e33, s="{:.1f}%".format(((core_significant < 0).sum() / len(core_significant)) * 100), ha="right", va="top", color="#01016f", fontsize=8)
ax.text(+0.01, y=10e33, s="{:.1f}%".format(((core_significant > 0).sum() / len(core_significant)) * 100), ha="left", va="top", color="#01016f", fontsize=8)

ax.text(-0.01, y=10e30, s="{:.1f}%".format(((hsp_significant < 0).sum() / len(hsp_significant)) * 100), ha="right", va="top", color="#d8031c", fontsize=8)
ax.text(+0.01, y=10e30, s="{:.1f}%".format(((hsp_significant > 0).sum() / len(hsp_significant)) * 100), ha="left", va="top", color="#d8031c", fontsize=8)

legend_elements = [Patch(facecolor='black', edgecolor='black',
                         label='Any\nn={}'.format(len(all_significant))),
                   Patch(facecolor='#8a8a8a', edgecolor='#8a8a8a',
                         label='Peripheral\nn={}'.format(len(peri_significant))),
                   Patch(facecolor='#01016f', edgecolor='#01016f',
                         label='Core Gene\nn={}'.format(len(core_significant))),
                   Patch(facecolor='#d8031c', edgecolor="#d8031c",
                         label='HSP\nn={}'.format(len(hsp_significant)))]


leg = ax.legend(handles=legend_elements, loc='upper left', title="Perturbagen", fontsize=6.8, title_fontsize=7, ncol=2, columnspacing=0.5, handletextpad=-0.5)

for patch in leg.get_patches():
    patch.set_height(15)
    patch.set_width(5)
    patch.set_y(-5)


ax.set_ylabel(r"$-\log(FDR)$")
ax.set_xlabel("Mean Differential Perturbation\n(Core Gene - Peripheral)")
#plt.tight_layout()
plt.savefig("Volcano_Overexpression_all.svg", bbox_inches="tight")


# Tests for overrepresentation of HSPs

In [ ]:
import numpy as np
p1 = 1
p2_1 = 0.3
p2_2 = 0.312


n1 = 4
n2_1 = 2339
n2_2 = 28

n2 = n2_1 + n2_2
p2 = ((n2_1 * p2_1) + (n2_2 * p2_2)) / n2


In [ ]:
z = (p1 - p2) / np.sqrt(0.33 * ( 1- 0.33) * ((1 / n1) + (1 / n2)))

In [ ]:
z

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

proportions_ztest([n1*p1, n2*p2], [n1, n2])

In [ ]:
p1 = 0.5
p2_1 = 0.331
p2_2 = 0.333


n1 = 2
n2_1 = 1532
n2_2 = 183

n2 = n2_1 + n2_2
p2 = ((n2_1 * p2_1) + (n2_2 * p2_2)) / n2

proportions_ztest([n1*p1, n2*p2], [n1, n2])

# Get the number of significant perturbagens across all traits

In [ ]:

for direction in ["knockdown", "overexpression"]:
    truths = []
    for trait in ["uc", "cad", "scz"]:
        df = pd.read_csv("/mnt/storage/cmap/2017/differential_perturbation_{}_{}_HT29.tsv".format(direction, trait), header=0, sep="\t")
        truths.append(df["FDR"] < 0.05)
    truths = np.asarray(truths).reshape((-1, 3))
    print((truths.sum(axis=1) > 0).sum()  / truths.shape[0])
    print(truths.shape[0])




In [ ]:
# heretic experiment: test if conserved position is only due to shared perturbagens

from speos.utils.config import Config
from speos.preprocessing.handler import InputHandler
import os 
os.chdir("..")
config = Config()
config.parse_yaml("/home/ubuntu/speos/config_uc_only_nohetio_film_newstorage.yaml")
prepro = InputHandler(config).get_preprocessor()
prepro.build_graph(adjacency=False)
os.chdir("notebooks")
uccore, other_coregenes, hsps,  noncore = get_coregenes("uc", prepro.id2hgnc.values())
racore, _, _,  _ = get_coregenes("ra", prepro.id2hgnc.values())


In [ ]:
df = get_differential_percentages(full_kd_df, uccore, hsps, noncore)[0]
(df.transpose().FDR < 0.05).sum() / len(df.transpose())

In [ ]:
df = get_differential_percentages(full_kd_df.loc[~full_kd_df.index.isin(uccore.intersection(racore)), :], uccore, hsps, noncore, alternative_core=uccore.difference(racore))[0]
(df.transpose().FDR < 0.05).sum() / len(df.transpose())

In [ ]:
len(uccore.difference(racore))

In [ ]:
df = get_differential_percentages(full_kd_df, uccore, hsps, noncore, alternative_core=uccore.intersection(racore))[0]
(df.transpose().FDR < 0.05).sum() / len(df.transpose())

In [ ]:
# muss zielgene rausnehmen die teil der core gene intesection intersection sind
df = get_differential_percentages(full_kd_df, uccore, hsps, noncore, alternative_core=racore.difference(uccore))[0]
(df.transpose().FDR < 0.05).sum() / len(df.transpose())

In [ ]:
df = get_differential_percentages(full_kd_df.loc[~full_kd_df.index.isin(uccore), :], uccore, hsps, noncore, alternative_core=racore.difference(uccore))[0]
(df.transpose().FDR < 0.05).sum() / len(df.transpose())

In [ ]:
len(full_kd_df)

In [ ]:
len(full_kd_df.loc[~full_kd_df.index.isin(uccore.intersection(racore)), :])

In [ ]:
len(racore.difference(uccore))

In [ ]:
df = get_differential_percentages(full_kd_df, uccore, hsps, noncore, alternative_core=uccore.intersection(sczcore))[0]
(df.transpose().FDR < 0.05).sum() / len(df.transpose())

In [ ]:
cadcore, _, _,  _ = get_coregenes("cad", prepro.id2hgnc.values())
adcore, _, _,  _ = get_coregenes("ad", prepro.id2hgnc.values())
sczcore, _, _,  _ = get_coregenes("scz", prepro.id2hgnc.values())

In [ ]:
df = get_differential_percentages(full_kd_df, uccore, hsps, noncore, alternative_core=adcore.difference(cadcore))[0]
(df.transpose().FDR < 0.05).sum() / len(df.transpose())

In [ ]:
df = get_differential_percentages(full_kd_df, uccore, hsps, noncore, alternative_core=sczcore.difference(adcore))[0]
(df.transpose().FDR < 0.05).sum() / len(df.transpose())

In [ ]:
df = get_differential_percentages(full_kd_df, uccore, hsps, noncore, alternative_core=sczcore)[0]
(df.transpose().FDR < 0.05).sum() / len(df.transpose())

In [ ]:
df = get_differential_percentages(full_kd_df, uccore, hsps, noncore, alternative_core=adcore)[0]
(df.transpose().FDR < 0.05).sum() / len(df.transpose())

In [ ]:
df = get_differential_percentages(full_kd_df, uccore, hsps, noncore, alternative_core=uccore.union(cadcore).union(sczcore))[0]
(df.transpose().FDR < 0.05).sum() / len(df.transpose())

In [ ]:
len(uccore.intersection(sczcore))


In [ ]:


df = get_differential_percentages(full_kd_df.loc[~full_kd_df.index.isin(uccore.intersection(racore)), :], uccore, hsps, noncore, alternative_core=uccore.difference(racore))[0]
(df.transpose().FDR < 0.05).sum() / len(df.transpose())

In [ ]:
import pandas as pd

df = pd.read_csv("/mnt/storage/cmap/2017/differential_perturbation_overexpression_gwas_uc_HT29.tsv", sep="\t", header=0, index_col=0)

In [ ]:
(df.FDR[df["Core Gene"]] < 0.05).sum() / df["Core Gene"].sum()

In [ ]:
(df.FDR[df["Peripheral"]] < 0.05).sum() / df["Peripheral"].sum()

In [ ]:
import numpy as np
labels = np.random.rand(5,5)

In [ ]:
labels

In [ ]:
transformed = labels.copy()

transformed[transformed > 0.5] = 1 - transformed[transformed > 0.5]

In [ ]:
transformed * 2

In [ ]:
np.minimum.reduce([labels, 1-labels]) * 2